In [ ]:
!pip install pyspark

In [ ]:
from pyspark import SparkConf, SparkContext

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [ ]:
#from pyspark.sql.types import StructField,IntegerType, StructType,StringType
#newDF=[StructField('ZXGID',IntegerType(),True),
#       StructField('ProductName',IntegerType(),True),
#       StructField('Sum_train',IntegerType(),True),
#       StructField('Sum_test',IntegerType(),True)]
#finalStruct=StructType(fields=newDF)
#sales_data_encoded_product = spark.read.csv('/content/drive/MyDrive/sales_data_encoded_product.csv', header='true',schema=finalStruct)

In [ ]:
sales_data_encoded_product = spark.read.option("inferSchema", True).csv('/content/drive/MyDrive/sales_data_encoded_product.csv', header='true')

In [ ]:
from pyspark.sql.types import IntegerType
sales_data_encoded_product = sales_data_encoded_product.withColumn("ZXGID", sales_data_encoded_product["ZXGID"].cast(IntegerType()))
sales_data_encoded_product = sales_data_encoded_product.withColumn("CombinedProductName", sales_data_encoded_product["CombinedProductName"].cast(IntegerType()))

In [ ]:
sales_data_encoded_product.printSchema()

root
 |-- ZXGID: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Industry: double (nullable = true)
 |-- UT L30 - Segment: string (nullable = true)
 |-- Family: string (nullable = true)
 |-- Product name: string (nullable = true)
 |-- PA part description: string (nullable = true)
 |-- Rev type: string (nullable = true)
 |-- 20161: double (nullable = true)
 |-- 20162: double (nullable = true)
 |-- 20163: double (nullable = true)
 |-- 20164: double (nullable = true)
 |-- 20171: double (nullable = true)
 |-- 20172: double (nullable = true)
 |-- 20173: double (nullable = true)
 |-- 20174: double (nullable = true)
 |-- 20181: double (nullable = true)
 |-- 20182: double (nullable = true)
 |-- 20183: double (nullable = true)
 |-- 20184: double (nullable = true)
 |-- 20191: double (nullable = true)
 |-- 20192: double (nullable = true)
 |-- 20193: double (nullable = true)
 |-- 20194: double (nullable = true)
 |-- 20201: double (nullable = true)
 |-- 20202: double (nullable

In [ ]:
sales_data_encoded_product.show(20)

+-----+-------------+--------+--------------------+------------+----------------+--------------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+
|ZXGID|      Country|Industry|    UT L30 - Segment|      Family|    Product name| PA part description|   Rev type|               20161|               20162|               20163|               20164|               2

In [ ]:
(training, test) = sales_data_encoded_product.randomSplit([0.8, 0.2])

In [ ]:
training.take(5)

[Row(ZXGID=0, Country='United States', Industry=99.0, UT L30 - Segment='Sterling B2B Integration', Family='STER FILEGWA', Product name='B2B Integrator', PA part description='IBM STERLING B2B INTEGRATOR BASIC EDITION PVU INSTL', Rev type='Transaction', 20161=0.0, 20162=0.0, 20163=0.005269563150756219, 20164=0.0, 20171=0.0, 20172=0.0, 20173=0.0, 20174=0.0, 20181=0.0, 20182=0.0, 20183=0.0, 20184=0.0, 20191=0.0, 20192=0.0, 20193=0.0, 20194=0.0, 20201=0.0, 20202=0.0, 20203=0.0, 20204=0.0, 20211=0.0, 20212=0.0, 20213=0.0, 20214=0.0, 20221=0.0, 20222=0.0, 20223=0.0, 20224=0.0, Grand Total=0.0008717558165409995, CombinedProductName=4, Sum_train=0.005269563150756219, Sum_test=0.0),
 Row(ZXGID=0, Country='United States', Industry=99.0, UT L30 - Segment='Sterling B2B Integration', Family='STER FILEGWA', Product name='B2B Integrator', PA part description='IBM STERLING B2B INTEGRATOR BASIC EDITION PVU SUB', Rev type='Subs & Supp', 20161=0.001221771231325215, 20162=0.0007654805498508604, 20163=0.001

In [ ]:
sales_data_encoded_product.select("UT L30 - Segment", "Family", "Product name", "CombinedProductName").show()

+--------------------+------------+----------------+-------------------+
|    UT L30 - Segment|      Family|    Product name|CombinedProductName|
+--------------------+------------+----------------+-------------------+
|Sterling B2B Inte...|STER FILEGWA|  B2B Integrator|                  4|
|Sterling B2B Inte...|STER FILEGWA|  B2B Integrator|                  4|
|Sterling B2B Inte...|STER FILEGWA|  B2B Integrator|                  4|
|Sterling B2B Inte...|STER FILEGWA|  B2B Integrator|                  4|
|Sterling B2B Inte...|STER FILEGWA|  B2B Integrator|                  4|
|Sterling B2B Inte...|STER FILEGWA|  B2B Integrator|                  4|
|Sterling B2B Inte...|STER FILEGWA|  B2B Integrator|                  4|
|Sterling B2B Inte...|STER FILEGWA|  B2B Integrator|                  4|
|Sterling B2B Inte...|STER FILEGWA|  B2B Integrator|                  4|
|Sterling B2B Inte...|STER FILEGWA|  B2B Integrator|                  4|
|Sterling B2B Inte...|STER FILEGWA|  B2B Integrator

In [ ]:
# Show unique UT L30 - Segement - Family - Product Name
sales_data_encoded_product_pd = sales_data_encoded_product.toPandas()
cols = ["UT L30 - Segment", "Family", "CombinedProductName"]
sales_data_encoded_product_pd['productInfoTemp'] = sales_data_encoded_product_pd[cols].apply(lambda row: '__'.join(row.values.astype(str)), axis=1)
cols_new = ["CombinedProductName","productInfoTemp"]
sales_data_encoded_product_pd['productInfo'] = sales_data_encoded_product_pd[cols_new].apply(lambda row: '#'.join(row.values.astype(str)), axis=1)


In [ ]:
#sales_data_encoded_product_pd[["productInfo", "ProductName"]].unique.head(5)
#sales_data_encoded_product_pd["productInfo", "ProductName"].unique.head(5)
import pandas as pd
unique = pd.unique(sales_data_encoded_product_pd[["productInfo"]].values.ravel('K'))
dict_productinfo = {}
for row in unique:
  str_info = row.split("#")
  dict_productinfo[str_info[0]] = str_info[1]
dict_productinfo

{'4': 'Sterling B2B Integration__STER FILEGWA__4',
 '27': 'Sterling B2B Integration__STER FILEGWA__27',
 '3': 'Sterling B2B Integration__STER FILEGWA__3',
 '40': 'Sterling B2B Integration__STER FILEGWA__40',
 '6': 'Sterling B2B Integration__STER FILEGWA__6',
 '58': 'Sterling Managed File Transfer__STER MFT__58',
 '66': 'Sterling Managed File Transfer__STER MFT__66',
 '65': 'Sterling Managed File Transfer__STER MFT__65',
 '61': 'Sterling Managed File Transfer__STER MFT__61',
 '160': 'Sterling Transformation Extender__STER TRANSX__160',
 '80': 'Sterling Managed File Transfer__STER MFT__80',
 '42': 'Sterling B2B Integration__STER FILEGWA__42',
 '16': 'Sterling B2B Integration__STER FILEGWA__16',
 '141': 'Sterling Transformation Extender__STER TRANSX__141',
 '173': 'Sterling Transformation Extender__STER TRANSX__173',
 '172': 'Sterling Transformation Extender__STER TRANSX__172',
 '171': 'Sterling Transformation Extender__STER TRANSX__171',
 '142': 'Sterling Transformation Extender__STER TR

In [ ]:
Product_list = sorted(dict_productinfo.items(), key=lambda x:int(x[0]))
Product_list

[('0', 'Sterling B2B Integration__STER FILEGWA__0'),
 ('1', 'Sterling B2B Integration__STER FILEGWA__1'),
 ('2', 'Sterling B2B Integration__STER FILEGWA__2'),
 ('3', 'Sterling B2B Integration__STER FILEGWA__3'),
 ('4', 'Sterling B2B Integration__STER FILEGWA__4'),
 ('5', 'Sterling B2B Integration__STER FILEGWA__5'),
 ('6', 'Sterling B2B Integration__STER FILEGWA__6'),
 ('7', 'Sterling B2B Integration__STER FILEGWA__7'),
 ('8', 'Sterling B2B Integration__STER FILEGWA__8'),
 ('9', 'Sterling B2B Integration__STER FILEGWA__9'),
 ('10', 'Sterling B2B Integration__STER FILEGWA__10'),
 ('11', 'Sterling B2B Integration__STER FILEGWA__11'),
 ('12', 'Sterling B2B Integration__STER FILEGWA__12'),
 ('13', 'Sterling B2B Integration__STER FILEGWA__13'),
 ('14', 'Sterling B2B Integration__STER FILEGWA__14'),
 ('15', 'Sterling B2B Integration__STER FILEGWA__15'),
 ('16', 'Sterling B2B Integration__STER FILEGWA__16'),
 ('17', 'Sterling B2B Integration__STER FILEGWA__17'),
 ('18', 'Sterling B2B Integrat

In [ ]:
# Show unique ZXGID
sales_data_encoded_product.select("ZXGID").distinct().show()

+-----+
|ZXGID|
+-----+
| 7253|
| 7993|
| 7982|
| 6654|
| 7240|
| 2366|
| 1238|
| 8389|
| 5518|
| 8086|
| 5300|
| 9427|
| 4818|
| 6620|
| 3918|
| 9900|
|10462|
|10703|
| 7417|
|10230|
+-----+
only showing top 20 rows



In [ ]:
# Show unique count of ZXGID # 10730
sales_data_encoded_product.select("ZXGID").distinct().count()

10730

In [ ]:
# Show unique count of CombinedProductName # At product level # 174
sales_data_encoded_product.select("CombinedProductName").distinct().count()

174

In [ ]:
# Show unique ProductName, Sum_Train, Sum_Test
sales_data_encoded_product.select("CombinedProductName", "Sum_Train", "Sum_Test").distinct().show()

+-------------------+--------------------+--------------------+
|CombinedProductName|           Sum_Train|            Sum_Test|
+-------------------+--------------------+--------------------+
|                 65|0.033670971240423814|0.004110994596881...|
|                 27| 0.01286897909773545|  0.1151311159052694|
|                150|0.001340032631711142|                 0.0|
|                 58| 0.19402172270091136|0.025465145939230903|
|                 66|0.053661381658842544|0.005856125221428004|
|                 58|0.001780500938184...|8.306560597770219E-5|
|                 65|0.022293740981078437|                 0.0|
|                 65|                 0.0|4.828865023564862E-6|
|                 84| 0.21378442799439348|                 0.0|
|                 66|5.619680193640614E-4|7.780478426578106E-4|
|                 65|0.003938043248665226|4.679362470077224E-4|
|                 58|0.030336366156146616|0.002832537163839...|
|                 65|2.919869912774737E-

In [ ]:
from pyspark.ml.recommendation import ALS

# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(rank=15, maxIter=10, regParam=0.01, userCol="ZXGID", itemCol="CombinedProductName", ratingCol="Sum_train",
          coldStartStrategy="drop", implicitPrefs=False)
model = als.fit(training)

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
predictions.show()
evaluator = RegressionEvaluator(metricName="rmse", labelCol="Sum_test",
                                predictionCol="prediction")

rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

+-----+-------------+--------+--------------------+------------+--------------------+--------------------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+------------+
|ZXGID|      Country|Industry|    UT L30 - Segment|      Family|        Product name| PA part description|   Rev type|               20161|               20162|               20163|               2

In [ ]:
# Generate 5 ProductName recommendations for each Company
ProductNameRecs = model.recommendForAllUsers(5)
ProductNameRecs.show(truncate=False)

+-----+-------------------------------------------------------------------------------------------------------+
|ZXGID|recommendations                                                                                        |
+-----+-------------------------------------------------------------------------------------------------------+
|1    |[{74, 0.013686378}, {50, 0.0101100495}, {51, 0.0093215285}, {76, 0.008020745}, {82, 0.0075783217}]     |
|3    |[{94, 1.3879292E-4}, {74, 1.22044985E-4}, {141, 1.1014774E-4}, {167, 1.0884059E-4}, {51, 9.0488065E-5}]|
|5    |[{74, 3.4257091E-4}, {141, 2.888227E-4}, {88, 2.0187373E-4}, {167, 1.8598979E-4}, {50, 1.7372606E-4}]  |
|6    |[{51, 0.0}, {49, 0.0}, {48, 0.0}, {47, 0.0}, {46, 0.0}]                                                |
|9    |[{74, 2.5857827E-5}, {141, 2.430303E-5}, {88, 1.7502864E-5}, {167, 1.4662458E-5}, {50, 1.2230183E-5}]  |
|12   |[{74, 0.020362217}, {50, 0.014698089}, {51, 0.013879289}, {76, 0.011272513}, {82, 0.010777711}]  

In [ ]:
ProductNameRecsdf = ProductNameRecs.toPandas()

In [ ]:
ProductNameRecsdf

,ZXGID,recommendations
0,1,"[(74, 0.013686377555131912), (50, 0.0101100495..."
1,3,"[(94, 0.0001387929223710671), (74, 0.000122044..."
2,5,"[(74, 0.0003425709146540612), (141, 0.00028882..."
3,6,"[(51, 0.0), (49, 0.0), (48, 0.0), (47, 0.0), (..."
4,9,"[(74, 2.585782749520149e-05), (141, 2.43030299..."
...,...,...
10072,10723,"[(74, 0.0005134196253493428), (50, 0.000410464..."
10073,10724,"[(74, 0.0005474775680340827), (141, 0.00046158..."
10074,10725,"[(74, 0.010854052379727364), (50, 0.0081258425..."
10075,10726,"[(74, 0.0023166658356785774), (50, 0.001734362..."


In [ ]:
#ProductNameRecsdf['recommendations']


In [ ]:
#result_array = [(47, 13.172346115112305), (65, 9.178628921508)]
import ast
def transform_result(result_array):
  tuple_list = []
  for result in result_array:
    dict_products = result.asDict()

    tuple_list.append(str(result['CombinedProductName']) + ","+str(result["rating"]))

  return "#".join(tuple_list)

#print(transform_result(result_array))

In [ ]:
ProductNameRecsdf["transformed_result"] = ProductNameRecsdf['recommendations'].apply(lambda x: transform_result(x))
ProductNameRecsdf.head(5)


,ZXGID,recommendations,transformed_result
0,1,"[(74, 0.013686377555131912), (50, 0.0101100495...","74,0.013686377555131912#50,0.01011004950851202..."
1,3,"[(94, 0.0001387929223710671), (74, 0.000122044...","94,0.0001387929223710671#74,0.0001220449848915..."
2,5,"[(74, 0.0003425709146540612), (141, 0.00028882...","74,0.0003425709146540612#141,0.000288822688162..."
3,6,"[(51, 0.0), (49, 0.0), (48, 0.0), (47, 0.0), (...","51,0.0#49,0.0#48,0.0#47,0.0#46,0.0"
4,9,"[(74, 2.585782749520149e-05), (141, 2.43030299...","74,2.585782749520149e-05#141,2.430302993161603..."


In [ ]:
import pandas as pd
ProductNameRecsdf[['Product1', 'Product2', 'Product3', 'Product4', 'Product5']] = ProductNameRecsdf["transformed_result"].apply(lambda x: pd.Series(str(x).split("#")))


for colName in ['Product1', 'Product2', 'Product3', 'Product4', 'Product5'] :
  ProductNameRecsdf[[colName, colName+"weight"]] = ProductNameRecsdf[colName].apply(lambda x: pd.Series(str(x).split(",")))

ProductNameRecsdf.head(5)

for colName in ['Product1', 'Product2', 'Product3', 'Product4', 'Product5'] :
  ProductNameRecsdf[colName +'Name'] = ProductNameRecsdf[colName].apply(lambda x: dict_productinfo[x])

ProductNameRecsdf.head(5)



,ZXGID,recommendations,transformed_result,Product1,Product2,Product3,Product4,Product5,Product1weight,Product2weight,Product3weight,Product4weight,Product5weight,Product1Name,Product2Name,Product3Name,Product4Name,Product5Name
0,1,"[(74, 0.013686377555131912), (50, 0.0101100495...","74,0.013686377555131912#50,0.01011004950851202...",74,50,51,76,82,0.013686377555131912,0.01011004950851202,0.009321528486907482,0.008020744659006596,0.007578321732580662,Sterling Managed File Transfer__STER MFT__74,Sterling Managed File Transfer__STER MFT__50,Sterling Managed File Transfer__STER MFT__51,Sterling Managed File Transfer__STER MFT__76,Sterling Partner Engagement Manager__PEM__82
1,3,"[(94, 0.0001387929223710671), (74, 0.000122044...","94,0.0001387929223710671#74,0.0001220449848915...",94,74,141,167,51,0.0001387929223710671,0.00012204498489154503,0.00011014773917850107,0.00010884059156524017,9.048806532518938e-05,Sterling Supply Chain Business Network__B2B CL...,Sterling Managed File Transfer__STER MFT__74,Sterling Transformation Extender__STER TRANSX_...,Sterling Transformation Extender__STER TRANSX_...,Sterling Managed File Transfer__STER MFT__51
2,5,"[(74, 0.0003425709146540612), (141, 0.00028882...","74,0.0003425709146540612#141,0.000288822688162...",74,141,88,167,50,0.0003425709146540612,0.0002888226881623268,0.00020187372865621,0.00018598978931549937,0.0001737260608933866,Sterling Managed File Transfer__STER MFT__74,Sterling Transformation Extender__STER TRANSX_...,Sterling Supply Chain Business Network__B2B CL...,Sterling Transformation Extender__STER TRANSX_...,Sterling Managed File Transfer__STER MFT__50
3,6,"[(51, 0.0), (49, 0.0), (48, 0.0), (47, 0.0), (...","51,0.0#49,0.0#48,0.0#47,0.0#46,0.0",51,49,48,47,46,0.0,0.0,0.0,0.0,0.0,Sterling Managed File Transfer__STER MFT__51,Sterling Managed File Transfer__STER MFT__49,Sterling B2B Integration__STER FILEGWA__48,Sterling B2B Integration__STER FILEGWA__47,Sterling B2B Integration__STER FILEGWA__46
4,9,"[(74, 2.585782749520149e-05), (141, 2.43030299...","74,2.585782749520149e-05#141,2.430302993161603...",74,141,88,167,50,2.585782749520149e-05,2.4303029931616038e-05,1.75028635567287e-05,1.4662457942904439e-05,1.223018261953257e-05,Sterling Managed File Transfer__STER MFT__74,Sterling Transformation Extender__STER TRANSX_...,Sterling Supply Chain Business Network__B2B CL...,Sterling Transformation Extender__STER TRANSX_...,Sterling Managed File Transfer__STER MFT__50


In [ ]:
Recommendation = ProductNameRecsdf[["ZXGID", "recommendations", "Product1Name", "Product1weight", "Product2Name", "Product2weight", "Product3Name",
                                    "Product3weight", "Product4Name", "Product4weight", "Product5Name", "Product5weight"]]
Recommendation.head(5)

,ZXGID,recommendations,Product1Name,Product1weight,Product2Name,Product2weight,Product3Name,Product3weight,Product4Name,Product4weight,Product5Name,Product5weight
0,1,"[(74, 0.013686377555131912), (50, 0.0101100495...",Sterling Managed File Transfer__STER MFT__74,0.013686377555131912,Sterling Managed File Transfer__STER MFT__50,0.01011004950851202,Sterling Managed File Transfer__STER MFT__51,0.009321528486907482,Sterling Managed File Transfer__STER MFT__76,0.008020744659006596,Sterling Partner Engagement Manager__PEM__82,0.007578321732580662
1,3,"[(94, 0.0001387929223710671), (74, 0.000122044...",Sterling Supply Chain Business Network__B2B CL...,0.0001387929223710671,Sterling Managed File Transfer__STER MFT__74,0.00012204498489154503,Sterling Transformation Extender__STER TRANSX_...,0.00011014773917850107,Sterling Transformation Extender__STER TRANSX_...,0.00010884059156524017,Sterling Managed File Transfer__STER MFT__51,9.048806532518938e-05
2,5,"[(74, 0.0003425709146540612), (141, 0.00028882...",Sterling Managed File Transfer__STER MFT__74,0.0003425709146540612,Sterling Transformation Extender__STER TRANSX_...,0.0002888226881623268,Sterling Supply Chain Business Network__B2B CL...,0.00020187372865621,Sterling Transformation Extender__STER TRANSX_...,0.00018598978931549937,Sterling Managed File Transfer__STER MFT__50,0.0001737260608933866
3,6,"[(51, 0.0), (49, 0.0), (48, 0.0), (47, 0.0), (...",Sterling Managed File Transfer__STER MFT__51,0.0,Sterling Managed File Transfer__STER MFT__49,0.0,Sterling B2B Integration__STER FILEGWA__48,0.0,Sterling B2B Integration__STER FILEGWA__47,0.0,Sterling B2B Integration__STER FILEGWA__46,0.0
4,9,"[(74, 2.585782749520149e-05), (141, 2.43030299...",Sterling Managed File Transfer__STER MFT__74,2.585782749520149e-05,Sterling Transformation Extender__STER TRANSX_...,2.4303029931616038e-05,Sterling Supply Chain Business Network__B2B CL...,1.75028635567287e-05,Sterling Transformation Extender__STER TRANSX_...,1.4662457942904439e-05,Sterling Managed File Transfer__STER MFT__50,1.223018261953257e-05


In [ ]:
# Generate top 5 company recommendations for each ProductName
#CompanyRecs = model.recommendForAllItems(5)
#CompanyRecs.show(5)